In [1]:
import numpy as np
import pandas as pd

/home/harshit/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/harshit/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
df_combined=pd.read_csv('mergedsharepriceanddf.csv')

In [3]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2761 entries, 0 to 2760
Columns: 155 entries, Unnamed: 0.1 to SharePrice
dtypes: float64(133), int64(6), object(16)
memory usage: 3.3+ MB


In [4]:
df_combined.head()

,Unnamed: 0.1,Unnamed: 0,symbol,calendarYear,date_x,reportedCurrency_x,cik_x,fillingDate_x,acceptedDate_x,period_x,...,othertotalStockholdersEquity,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,SharePrice
0,0,0,AACG,2020,2019-12-31,CNY,1420529,2020-04-28,2020-04-28 16:10:47,FY,...,533076993.0,300239664.0,350630935.0,667654163.0,50391271.0,667654163.0,45726391.0,25547017.0,-128650741.0,1.19
1,1,1,AACG,2021,2020-12-31,CNY,1420529,2021-04-13,2021-04-13 06:13:21,FY,...,529646579.0,198404863.0,247746232.0,575621032.0,49341369.0,575621032.0,44000000.0,47778952.0,-64944481.0,1.07
2,2,2,AACG,2022,2021-12-31,CNY,1420529,2022-04-26,2022-04-26 08:55:43,FY,...,530764810.0,187769092.0,190899534.0,507175310.0,3130442.0,507175310.0,38000000.0,40717267.0,-30622094.0,1.27
3,3,3,AACG,2023,2022-12-31,CNY,1420529,2023-04-12,2023-04-12 06:10:25,FY,...,533431198.0,143099333.0,143514190.0,474464769.0,414857.0,474464769.0,38000000.0,36449192.0,-18531007.0,1.16
4,4,5,AACI,2021,2021-09-30,USD,1844817,2021-12-29,2021-12-29 17:23:59,FY,...,1378693.0,150910477.0,150910477.0,151119504.0,0.0,151119504.0,150001052.0,0.0,-657590.0,9.79


In [5]:
columns=df_combined.columns.tolist()

In [6]:
for column in columns:
    if df_combined[column].dtype==object and column!='symbol':
        df_combined.drop(columns=[column],inplace=True)
        print(f"removed {column}")

removed date_x
removed reportedCurrency_x
removed fillingDate_x
removed acceptedDate_x
removed period_x
removed date_y
removed reportedCurrency_y
removed fillingDate_y
removed acceptedDate_y
removed period_y
removed date
removed reportedCurrency
removed fillingDate
removed acceptedDate
removed period


## Finding change in each year for better understandin

In [7]:
df_combined.drop(columns=['Unnamed: 0.1',
 'Unnamed: 0'],inplace=True)

In [8]:
df=df_combined

## getting all possible changes...in columns

In [9]:


# Assuming df is your original DataFrame
# Filter out the year 2023

# Get all unique years excluding 2023
years = sorted(df['calendarYear'].unique())

# Create all possible combinations of years (excluding 2023)
year_combinations = [(y2, y1) for i, y1 in enumerate(years) for y2 in years[i+1:]]

# Initialize a dictionary to store the differences
diff_data = {}

# Calculate the differences for each combination
for y2, y1 in year_combinations:
    df_y2 = df[df['calendarYear'] == y2].reset_index(drop=True)
    df_y1 = df[df['calendarYear'] == y1].reset_index(drop=True)
    numeric_cols = df.select_dtypes(include='number').columns
    diff = df_y2[numeric_cols] - df_y1[numeric_cols]
    diff['year_diff'] = f"{y2}-{y1}"
    diff_data[f"{y2}-{y1}"] = diff

# Concatenate all the differences into a single DataFrame
diff_df = pd.concat(diff_data.values(), keys=diff_data.keys()).reset_index(level=1, drop=True)

# If you want to save the result to a CSV file
diff_df.to_csv('yearly_differences.csv', index=False)


In [10]:
import pandas as pd

# Load your original DataFrame (assuming it's called df)
# df = pd.read_csv('your_dataframe.csv')  # Uncomment and modify to load your actual DataFrame

# Get all unique years excluding 2023
years = sorted(df['calendarYear'].unique())

# Create all possible combinations of years (excluding 2023)
year_combinations = [(y2, y1) for i, y1 in enumerate(years) for y2 in years[i+1:]]

# Initialize a dictionary to store the differences
diff_data = {}

# Calculate the differences for each combination
for y2, y1 in year_combinations:
    df_y2 = df[df['calendarYear'] == y2].reset_index(drop=True)
    df_y1 = df[df['calendarYear'] == y1].reset_index(drop=True)
    numeric_cols = df.select_dtypes(include='number').columns
    
    # Calculate the differences for the numeric columns
    diff = df_y2[numeric_cols] - df_y1[numeric_cols]
    
    # Add the share prices for y2 and y1
    diff['SharePrice_y2'] = df_y2['SharePrice']
    diff['SharePrice_y1'] = df_y1['SharePrice']
    
    # Add the year difference identifier
    diff['year_diff'] = f"{y2}-{y1}"
    
    # Store the difference data
    diff_data[f"{y2}-{y1}"] = diff

# Concatenate all the differences into a single DataFrame
diff_df = pd.concat(diff_data.values(), keys=diff_data.keys()).reset_index(level=1, drop=True)

# If you want to save the result to a CSV file
diff_df.to_csv('yearly_differences2.csv', index=False)

print(diff_df)


           calendarYear      cik_x  cashAndCashEquivalents_x  \
2020-2019           1.0    -1613.0               115673222.0   
2020-2019           1.0  1415941.0              -275545000.0   
2020-2019           1.0    -1976.0               232107000.0   
2020-2019           1.0 -1149937.0               -47709000.0   
2020-2019           1.0   333972.0                16628000.0   
...                 ...        ...                       ...   
2023-2022           NaN        NaN                       NaN   
2023-2022           NaN        NaN                       NaN   
2023-2022           NaN        NaN                       NaN   
2023-2022           NaN        NaN                       NaN   
2023-2022           NaN        NaN                       NaN   

           shortTermInvestments_x  cashAndShortTermInvestments_x  \
2020-2019            0.000000e+00                   1.156732e+08   
2020-2019           -3.546000e+09                  -3.821545e+09   
2020-2019            6.3865

In [11]:
diff_df.columns.tolist()

['calendarYear',
 'cik_x',
 'cashAndCashEquivalents_x',
 'shortTermInvestments_x',
 'cashAndShortTermInvestments_x',
 'netReceivables_x',
 'inventory_x',
 'otherCurrentAssets_x',
 'totalCurrentAssets_x',
 'propertyPlantEquipmentNet_x',
 'goodwill_x',
 'intangibleAssets_x',
 'goodwillAndIntangibleAssets_x',
 'longTermInvestments_x',
 'taxAssets_x',
 'otherNonCurrentAssets_x',
 'totalNonCurrentAssets_x',
 'otherAssets_x',
 'totalAssets_x',
 'accountPayables_x',
 'shortTermDebt_x',
 'taxPayables_x',
 'deferredRevenue_x',
 'otherCurrentLiabilities_x',
 'totalCurrentLiabilities_x',
 'longTermDebt_x',
 'deferredRevenueNonCurrent_x',
 'deferredTaxLiabilitiesNonCurrent_x',
 'otherNonCurrentLiabilities_x',
 'totalNonCurrentLiabilities_x',
 'otherLiabilities_x',
 'capitalLeaseObligations_x',
 'totalLiabilities_x',
 'preferredStock_x',
 'commonStock_x',
 'retainedEarnings_x',
 'accumulatedOtherComprehensiveIncomeLoss_x',
 'othertotalStockholdersEquity_x',
 'totalStockholdersEquity_x',
 'totalEqui